In [1]:
import numpy as np
import pandas as pd
import pickle


In [ ]:
import json
import re
import sys
import time
import warnings
from random import sample

import openai
from openai import OpenAI
import pandas as pd
# from tqdm import tqdm
import tiktoken
key = "insert_key_here"

client = OpenAI(api_key= key, base_url="https://api.deepseek.com")
# Parameters

# Function
def get_response(message, instruction):
    response = client.chat.completions.create(
		model = 'deepseek-reasoner',
        temperature = 1,
        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": message}
        ]
    )
    
    # print token usage
    print(response.usage)
    # return the response
    return response.choices[0].message.content
  
def LM_generation(prompt, max_tokens = 10, temperature = 0.1, model = "deepseek-reasoner", system_msg = None, msg_history = None):
    retry_attempt = 5
    retry_num = 0
    generation_success = False
    messages = []
    if system_msg is not None:
        messages.append({'role':'system', 'content': system_msg})
    if msg_history is not None:
        messages = msg_history
    messages.append({'role':'user', 'content':prompt})
    while retry_num < retry_attempt and not generation_success:
        try:
            gen = client.chat.completions.create(
                model = model,
                messages = messages,
                max_tokens = max_tokens,
                temperature = temperature,
                stream = False
            ) 
            generation_success = True
            input_tokens = gen.usage.prompt_tokens
            output_tokens = gen.usage.completion_tokens
        except openai.APIError as e:
            retry_num += 1
            generation_success = False
            time.sleep(5)
            # return "openai.error.APIError", -1, -1 
        except openai.RateLimitError as e:
            retry_num += 1
            generation_success = False
            time.sleep(30)
        except:
            retry_num += 1
            generation_success = False
            time.sleep(10)
    
    if generation_success:
        if gen == None or gen.choices[0] == None or gen.choices[0].message == None or gen.choices[0].message.content == None:
            return "", 0, 0
        # return gen.choices[0].message.content.strip(), input_tokens, output_tokens
        return gen, input_tokens, output_tokens
    else:
        return "Error", -1,-1

def count_tokens(text, encoding_name = "cl100k_base"):
    encoding = tiktoken.get_encoding(encoding_name)
    # print(text)
    num_tokens = len(encoding.encode(text))
    return num_tokens


In [ ]:
import json

# Open and read the JSON file
with open('llm_ready_opentom_vfin.jsonl') as f: # ambiguous story
    final_opentom = [json.loads(line) for line in f]
    
with open('llm_ready_bigtom_vfin.jsonl') as f: # ambiguous story
    final_bigtom = [json.loads(line) for line in f]

bigtom_filter = np.load('bigtom_filter.npy', allow_pickle=True)
opentom_filter = np.load('opentom_filter.npy', allow_pickle=True)

final_opentom = np.array(final_opentom)[opentom_filter]
final_bigtom = np.array(final_bigtom)[bigtom_filter]

In [4]:
final_bigtom[0]

{'ID': '0-0-0',
 'category': 'pos/hard_neg',
 'sense': 'visual',
 'story': 'Noor is working as a barista at a busy coffee shop.  Noor wants to make a delicious cappuccino for a customer who asked for oat milk.  Noor grabs a milk pitcher and fills it with oat milk.',
 'event': " A coworker, who didn't hear the customer's request, swaps the oat milk in the pitcher with almond milk while Noor is attending to another task.",
 'protag_name': 'Noor',
 'percept_cue': "As Noor pours the milk into the espresso, she notices the liquid's texture. Instead of the familiar creamy, smooth flow that oat milk typically has, the milk appears slightly more translucent, with a thinner consistency that swirls differently in the cup, creating delicate, lacy patterns.",
 'initial_belief': ' Noor believes that the milk pitcher contains oatmilk.',
 'default_question': "After the event happened, based on the main character Noor's perception of the event, which of the following statements best describes Noor's n

In [5]:
#save

def save_list_of_dict(save_name, list_of_dict):
    with open(save_name, 'wb') as f:
        pickle.dump(list_of_dict, f)

def load_list_of_dict(save_name):
    with open(save_name, 'rb') as f:
        loaded = pickle.load(f)
    return loaded

In [ ]:
senses = ['visual', 'auditory', 'olfactory', 'gustatory', 'tactile', 'proprio']
num_senses = len(senses)
# thermos can be merged with tactile, but minor issue.

In [7]:
sense_dict = {senses[i]: i for i in range(num_senses)}

LLM Task 1:

In [8]:
# Answer Holder - I think we should use list? Better to store answer, then process later.
# Can hold answers in a list of dict
# {ID, groundtruth_type, sense, option_order (B/A), llm_answer, true_answer}

In [8]:
bigtom_task1_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [[0,1], [1,0],[0,1],[1,0]], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X'],
                     'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']],
                     'reasoning_content': ['X','X','X','X']} 
                    for instance in final_bigtom]
opentom_task1_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [[0,1], [1,0],[0,1],[1,0]], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X'],
                     'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']],
                     'reasoning_content': ['X','X','X','X']} 
                    for instance in final_opentom]

In [9]:
# Prompts
bigtom_system = "Story: {story} \n\nEvent: {event} \n\n"
bigtom_system += "Regarding the event, the main character {protag_name} only encounters the following percept: {percept}\n\n"
bigtom_system += "Before the event, the main character's initial belief was \"{initial_belief}\"\n\n"
bigtom_system += "Instructions: Analyze the situation and select the most appropriate answer to the following question: \n\n\n"

opentom_system = "Story: {story} \n\n"
opentom_system += "After the story, the main character {protag_name} only encounters the following percept: {percept}\n\n"
opentom_system += "Before encountering the percept, the main character's initial belief was \"{initial_belief}\"\n\n"
opentom_system += "Instructions: Analyze the situation and select the most appropriate answer to the following question: \n\n\n"


# Needed variables: story, event, protag_name, percept, initial_belief

In [10]:
bigtom_q1_prompt_1 = "After the event happened, based on the main character {protag_name} perception of the event, "
bigtom_q1_prompt_1 += "is the following statement about {protag_name}'s belief True or False: \n"
bigtom_q1_prompt_1 += "Statement: \"{initial_belief}\"\n"

bigtom_q1 = [bigtom_q1_prompt_1, bigtom_q1_prompt_1]
bigtom_q1[0] = bigtom_q1[0] + "A. True\nB. False"
bigtom_q1[1] = bigtom_q1[1] + "A. False\nB. True"

bigtom_q1_prompt_2 = "Did the event cause the main character {protag_name} to change their belief?\n"
bigtom_q1.append(bigtom_q1_prompt_2 + "A. No\nB. Yes")
bigtom_q1.append(bigtom_q1_prompt_2 + "A. Yes\nB. No")

opentom_q1_prompt_1 = "After encountering the percept, based on the main character {protag_name}'s perception, is the following statement about Amir's belief True or False: \n"
opentom_q1_prompt_1 += "Statement: \"{initial_belief}\"\n"
opentom_q1 = [opentom_q1_prompt_1, opentom_q1_prompt_1]
opentom_q1[0] = opentom_q1[0] + "A. True\nB. False"
opentom_q1[1] = opentom_q1[1] + "A. False\nB. True"

opentom_q1_prompt_2 = "Did the percept cause the main character {protag_name} to change their belief?\n"
opentom_q1.append(opentom_q1_prompt_2 + "A. No\nB. Yes")
opentom_q1.append(opentom_q1_prompt_2 + "A. Yes\nB. No")

# Needed variable: protag_name, initial_belief

In [11]:
option_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

# def post_randomize_order(val, n): 
#     # val = 
#     # n = number of randomize

bigtom_task2_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X'],
                     'true_answer': [],
                     'true_answer_og': option_dict[instance['new_belief_label']],
                     'reasoning_content': ['X','X','X','X','X']} 
                    for instance in final_bigtom]
opentom_task2_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X'],
                     'true_answer': [],
                     'true_answer_og': option_dict[instance['new_belief_label']],
                     'reasoning_content': ['X','X','X','X','X']}  
                    for instance in final_opentom]

In [12]:
import random
random.seed(0)
np.random.seed(0)
task2_ans = [bigtom_task2_ans, opentom_task2_ans]
for t in task2_ans:
    for instance in t:
        for _ in range(5):
            randomizer = np.random.choice(np.arange(4), 4, replace=False) # 0 --> a0, 1 --> a1
            instance['answer_order'].append(randomizer.tolist())
            # instance['true_answer'].append(['A','B','C','D'][randomizer[instance['true_answer_og']]])
            true_answer_idx = np.where(randomizer == instance['true_answer_og'])[0][0]
            instance['true_answer'].append(['A','B','C','D'][true_answer_idx])

In [13]:
bigtom_q2_prompt = "{default_question}\n"
bigtom_q2_prompt += "Choose the most appropriate answer (choose one only):\n"
bigtom_q2_prompt += "A. {first}\nB. {second}\nC. {third}\nD. {fourth}"
opentom_q2_prompt = bigtom_q2_prompt

# "After the event happened, based on the main character Noor's perception of the event, which of the following statements best describes Noor's new belief?"

In [14]:
import json

def save_jsonl(file_name, path):
    with open(path, "w") as file:
        for item in file_name:
            json_string = json.dumps(item)
            file.write(json_string + '\n')


In [15]:
# Choose model
model = "deepseek-reasoner"

In [ ]:
# BigToM first - Task 1
model = "deepseek-reasoner"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()


for i in range(len(bigtom_task1_ans)): # set to len(bigtom_task1_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(4):
    # for k in range(4):
        #story, event, protag_name, percept, initial_belief
        story = final_bigtom[i]['story']
        event = final_bigtom[i]['event']
        protag_name = final_bigtom[i]['protag_name']
        percept = final_bigtom[i]['percept_cue']
        initial_belief = final_bigtom[i]['initial_belief']
        prompt = bigtom_system.format(story = story, event = event, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
        
        prompt += bigtom_q1[k].format(protag_name = protag_name, initial_belief = initial_belief)
        prompt += "\nProvide the answer as a single letter:"

        input_tokens += count_tokens(prompt)
        
        # print(prompt)
        # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
        
        
        # Add back later
        output = LM_generation(prompt, temperature = 0.5, max_tokens = 3, model = model)
        
        # Printouts - Remove later:
        output = output[0]
        reasoning_content = output.choices[0].message.reasoning_content
        llm_answer = output.choices[0].message.content
        llm_answer = llm_answer[0]
        # print(reasoning_content)
        # print(content)
        
        bigtom_task1_ans[i]['reasoning_content'][k] = reasoning_content
        bigtom_task1_ans[i]['llm_answer'][k] = llm_answer

                
        output_tokens += count_tokens(llm_answer)
        save_jsonl(bigtom_task1_ans, f'bigtom_task1_ans_{model}.jsonl')
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)

bigtom_task1_llm_answer = [instance['llm_answer'] for instance in bigtom_task1_ans]
bigtom_task1_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in bigtom_task1_ans]
bigtom_task1_llm_answer = np.array(bigtom_task1_llm_answer)
bigtom_task1_true_answer = np.array(bigtom_task1_true_answer)
bigtom_task1_accuracy = (bigtom_task1_llm_answer == bigtom_task1_true_answer).mean(axis=1)

gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
bigtom_task1_correct = np.zeros((100, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
bigtom_task1_total = np.zeros((100, 3, 8))
for i, instance in enumerate(bigtom_task1_ans):
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    bigtom_task1_total[story_idx][gt_idx][sense_idx] = 1
    bigtom_task1_correct[story_idx][gt_idx][sense_idx] = bigtom_task1_accuracy[i]
eps = 1e-10
print("BigToM Task 1 TPR, TNR-hard, TNR-easy: ", bigtom_task1_correct.sum(axis = 0).sum(axis = 1)/(bigtom_task1_total.sum(axis = 0).sum(axis = 1) + eps))


# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

for i in range(len(opentom_task1_ans)): # set to len(bigtom_task1_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(4):
        #story, event, protag_name, percept, initial_belief
        story = final_opentom[i]['story']
        protag_name = final_opentom[i]['protag_name']
        percept = final_opentom[i]['percept_cue']
        initial_belief = final_opentom[i]['initial_belief']
        prompt = opentom_system.format(story = story, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
        
        prompt += opentom_q1[k].format(protag_name = protag_name, initial_belief = initial_belief)

        input_tokens += count_tokens(prompt)
        
        # print(prompt)
        # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
        
        
        # Add back later
        output = LM_generation(prompt, temperature = 0.5, max_tokens = 3, model = model)
        
        # Printouts - Remove later:
        output = output[0]
        reasoning_content = output.choices[0].message.reasoning_content
        llm_answer = output.choices[0].message.content
        llm_answer = llm_answer[0]
        # print(reasoning_content)
        # print(content)
        
        bigtom_task1_ans[i]['reasoning_content'][k] = reasoning_content
        bigtom_task1_ans[i]['llm_answer'][k] = llm_answer

                
        output_tokens += count_tokens(llm_answer)
        save_jsonl(opentom_task1_ans, f'opentom_task1_ans_{model}.jsonl')
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)




0-th story


In [ ]:
# # Scoring
# # Array: [story, groundtruth, sense] - correct & total

# opentom_task1_llm_answer = [instance['llm_answer'] for instance in opentom_task1_ans]
# opentom_task1_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in opentom_task1_ans]
# opentom_task1_llm_answer = np.array(opentom_task1_llm_answer)
# opentom_task1_true_answer = np.array(opentom_task1_true_answer)
# opentom_task1_accuracy = (opentom_task1_llm_answer == opentom_task1_true_answer).mean(axis=1)
# gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
# opentom_task1_correct = np.zeros((121, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
# opentom_task1_total = np.zeros((121, 3, 8))
# for i, instance in enumerate(opentom_task1_ans):
#     ider = instance['ID']
#     story_idx = int(ider.split('-')[0])
#     gt_idx = gt_id[instance['groundtruth_classification']]
#     sense_idx = sense_dict[instance['sense']]
#     opentom_task1_total[story_idx][gt_idx][sense_idx] = 1
#     opentom_task1_correct[story_idx][gt_idx][sense_idx] = opentom_task1_accuracy[i]
    

In [ ]:
# eps = 1e-10
# print("TPR, TNR-hard, TNR-easy: ", opentom_task1_correct.sum(axis = 0).sum(axis = 1)/(opentom_task1_total.sum(axis = 0).sum(axis = 1) + eps))

TPR, TNR-hard, TNR-easy:  [0.80837264 0.44670051 0.72932692]


Done

Task 2

In [ ]:
# BigToM first - Task 1
model = "deepseek-reasoner"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()


for i in range(len(bigtom_task2_ans)): # set to len(bigtom_task1_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(5):
        #story, event, protag_name, percept, initial_belief
        story = final_bigtom[i]['story']
        event = final_bigtom[i]['event']
        protag_name = final_bigtom[i]['protag_name']
        percept = final_bigtom[i]['percept_cue']
        initial_belief = final_bigtom[i]['initial_belief']
        prompt = bigtom_system.format(story = story, event = event, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
        
        default_question = final_bigtom[i]['default_question']
        randomizer = np.array(bigtom_task2_ans[i]['answer_order'][k])
        # print(final_bigtom[i]['answer_options'])
        # print(randomizer)
        first = final_bigtom[i]['answer_options'][randomizer[0]]
        second = final_bigtom[i]['answer_options'][randomizer[1]]
        third = final_bigtom[i]['answer_options'][randomizer[2]]
        fourth = final_bigtom[i]['answer_options'][randomizer[3]]
        prompt += bigtom_q2_prompt.format(default_question=default_question, first=first, second=second, third=third, fourth=fourth)
        prompt += "\nProvide the answer as a single letter:"
        
        input_tokens += count_tokens(prompt)
        
        # print(prompt)
        # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
        
        
        # Add back later
        output = LM_generation(prompt, temperature = 0.5, max_tokens = 3, model = model)
        
        # Printouts - Remove later:
        output = output[0]
        reasoning_content = output.choices[0].message.reasoning_content
        llm_answer = output.choices[0].message.content
        llm_answer = llm_answer[0]
        # print(reasoning_content)
        # print(content)
        
        bigtom_task2_ans[i]['reasoning_content'][k] = reasoning_content
        bigtom_task2_ans[i]['llm_answer'][k] = llm_answer

                
        output_tokens += count_tokens(llm_answer)
        save_jsonl(bigtom_task2_ans, f'bigtom_task2_ans_{model}.jsonl')
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)

bigtom_task2_llm_answer = [instance['llm_answer'] for instance in bigtom_task2_ans]
bigtom_task2_true_answer = [instance['true_answer'] for instance in bigtom_task2_ans]
# bigtom_task2_llm_answer = bigtom_task2_llm_answer[0:50]
# bigtom_task2_true_answer = bigtom_task2_true_answer[0:50]
bigtom_task2_llm_answer = np.array(bigtom_task2_llm_answer)
bigtom_task2_true_answer = np.array(bigtom_task2_true_answer)
bigtom_task2_accuracy = (bigtom_task2_llm_answer == bigtom_task2_true_answer).mean(axis=1)

gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
bigtom_task2_correct = np.zeros((100, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
bigtom_task2_total = np.zeros((100, 3, 8))
for i, instance in enumerate(bigtom_task2_ans):
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    bigtom_task2_total[story_idx][gt_idx][sense_idx] = 1
    bigtom_task2_correct[story_idx][gt_idx][sense_idx] = bigtom_task2_accuracy[i]
eps = 1e-10
print("TPR, TNR-hard, TNR-easy: ", bigtom_task2_correct.sum(axis = 0).sum(axis = 1)/(bigtom_task2_total.sum(axis = 0).sum(axis = 1) + eps))

# OpenToM first - Task 1
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# for i in range(2):
for i in range(len(opentom_task2_ans)): # set to len(bigtom_task1_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(5):
        #story, event, protag_name, percept, initial_belief
        story = final_opentom[i]['story']
        protag_name = final_opentom[i]['protag_name']
        percept = final_opentom[i]['percept_cue']
        initial_belief = final_opentom[i]['initial_belief']
        prompt = opentom_system.format(story = story, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
        
        default_question = final_opentom[i]['default_question']
        randomizer = np.array(opentom_task2_ans[i]['answer_order'][k])
        # print(final_bigtom[i]['answer_options'])
        # print(randomizer)
        first = final_opentom[i]['answer_options'][randomizer[0]]
        second = final_opentom[i]['answer_options'][randomizer[1]]
        third = final_opentom[i]['answer_options'][randomizer[2]]
        fourth = final_opentom[i]['answer_options'][randomizer[3]]
        prompt += opentom_q2_prompt.format(default_question=default_question, first=first, second=second, third=third, fourth=fourth)
        prompt += "\nProvide the answer as a single letter:"
        
        input_tokens += count_tokens(prompt)
        
        # print(prompt)
        # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
        
        
        # Add back later
        output = LM_generation(prompt, temperature = 0.5, max_tokens = 3, model = model)
        
        # Printouts - Remove later:
        output = output[0]
        reasoning_content = output.choices[0].message.reasoning_content
        llm_answer = output.choices[0].message.content
        llm_answer = llm_answer[0]
        # print(reasoning_content)
        # print(content)
        
        opentom_task2_ans[i]['reasoning_content'][k] = reasoning_content
        opentom_task2_ans[i]['llm_answer'][k] = llm_answer

                
        output_tokens += count_tokens(llm_answer)
        save_jsonl(opentom_task2_ans, f'opentom_task2_ans_{model}.jsonl')
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)

opentom_task2_llm_answer = [instance['llm_answer'] for instance in opentom_task2_ans]
opentom_task2_true_answer = [instance['true_answer'] for instance in opentom_task2_ans]
opentom_task2_llm_answer = np.array(opentom_task2_llm_answer)
opentom_task2_true_answer = np.array(opentom_task2_true_answer)
opentom_task2_accuracy = (opentom_task2_llm_answer == opentom_task2_true_answer).mean(axis=1)
gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
opentom_task2_correct = np.zeros((121, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
opentom_task2_total = np.zeros((121, 3, 8))
for i, instance in enumerate(opentom_task2_ans):
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    opentom_task2_total[story_idx][gt_idx][sense_idx] = 1
    opentom_task2_correct[story_idx][gt_idx][sense_idx] = opentom_task2_accuracy[i]
eps = 1e-10
print("TPR, TNR-hard, TNR-easy: ", opentom_task2_correct.sum(axis = 0).sum(axis = 1)/(opentom_task2_total.sum(axis = 0).sum(axis = 1) + eps))


0-th story


Processing for Tasks 3 and 4

In [16]:
final_bigtom_with_noise = []
bigtom_noise_percepts = {i:[] for i in range(100)}
final_opentom_with_noise = []
opentom_noise_percepts = {i:[] for i in range(121)}
random.seed(0)
for instance in final_bigtom:
    if instance['groundtruth_classification'] == 'easy_negative':
        # find ID first
        story_idx = instance['ID'].split('-')[0]
        story_idx = int(story_idx)
        bigtom_noise_percepts[story_idx].append(instance['percept_cue'])
    else:
        final_bigtom_with_noise.append(instance)
# print(bigtom_noise_percepts[0])

rem_list = []
for instance in final_bigtom_with_noise:
    story_idx = instance['ID'].split('-')[0]
    story_idx = int(story_idx)
    # put the noise into the dataset
    instance['noise'] = []
    # if (len(bigtom_noise_percepts[story_idx])==0):
    #     print(instance)
    # if story_idx == 91:
    #     print(instance['ID'])
    for _ in range(5):
        if len(bigtom_noise_percepts[story_idx]) == 0:
            # print(instance['ID'])
            # final_bigtom_with_noise.remove(instance)
            rem_list.append(instance)
            break
        else:
            # print((bigtom_noise_percepts[story_idx]))
            instance['noise'].append(random.choice(bigtom_noise_percepts[story_idx]))
for story_idx in range(100):
    if len(bigtom_noise_percepts[story_idx]) == 0:
        del bigtom_noise_percepts[story_idx]
for instance in rem_list:
    final_bigtom_with_noise.remove(instance)
            
for instance in final_opentom:
    if instance['groundtruth_classification'] == 'easy_negative':
        # find ID first
        story_idx = instance['ID'].split('-')[0]
        story_idx = int(story_idx)
        opentom_noise_percepts[story_idx].append(instance['percept_cue'])
    else:
        final_opentom_with_noise.append(instance)
# print(bigtom_noise_percepts[0])
rem_list = []
for instance in final_opentom_with_noise:
    story_idx = instance['ID'].split('-')[0]
    story_idx = int(story_idx)
    # put the noise into the dataset
    instance['noise'] = []
    # if (len(bigtom_noise_percepts[story_idx])==0):
    #     print(instance)
    for _ in range(5):
        if len(opentom_noise_percepts[story_idx]) == 0:
            # final_opentom_with_noise.remove(instance)
            rem_list.append(instance)
            break
        else:
            # print((bigtom_noise_percepts[story_idx]))
            instance['noise'].append(random.choice(opentom_noise_percepts[story_idx]))
            
for story_idx in range(121):
    if len(opentom_noise_percepts[story_idx]) == 0:
        del opentom_noise_percepts[story_idx]
for instance in rem_list:
    final_bigtom_with_noise.remove(instance)

In [17]:

import json
with open("llm_ready_bigtom_with_noise.jsonl", "w") as file:
    for item in final_bigtom_with_noise:
        json_string = json.dumps(item)
        file.write(json_string + '\n')

with open("llm_ready_opentom_with_noise.jsonl", "w") as file:
    for item in final_opentom_with_noise:
        json_string = json.dumps(item)
        file.write(json_string + '\n')

[]

Task 3

In [ ]:
bigtom_task3_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [[0,1], [1,0],[0,1],[1,0], [0,1], [1,0],[0,1],[1,0]], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X','X','X','X'],
                     'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']],
                     'reasoning_content': ['X','X','X','X','X','X','X','X']} 
                    for instance in final_bigtom_with_noise]
opentom_task3_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [[0,1], [1,0],[0,1],[1,0], [0,1], [1,0],[0,1],[1,0]], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X','X','X','X'],
                     'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']],
                     'reasoning_content': ['X','X','X','X','X','X','X','X']} 
                    for instance in final_opentom_with_noise]


In [ ]:
# BigToM first - Task 3
model = "deepseek-reasoner"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# for i in range(11,50):
for i in range(len(bigtom_task3_ans)): # set to len(bigtom_task3_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(4):
        #story, event, protag_name, percept, initial_belief
        for j in range(2):# percept_order
            story = final_bigtom_with_noise[i]['story']
            event = final_bigtom_with_noise[i]['event']
            protag_name = final_bigtom_with_noise[i]['protag_name']
            percept_signal = final_bigtom_with_noise[i]['percept_cue']
            # in each iteration, pick a different noise
            percept_noise = final_bigtom_with_noise[i]['noise'][k]
            
            if (j==0):
                percept = f"\n1. {percept_signal} \n2. {percept_noise}\n"
            else:
                percept = f"\n1. {percept_noise} \n2. {percept_signal}\n"
            
            
            initial_belief = final_bigtom_with_noise[i]['initial_belief']
            prompt = bigtom_system.format(story = story, event = event, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
            
            prompt += bigtom_q1[k].format(protag_name = protag_name, initial_belief = initial_belief)
            prompt += "\nProvide the answer as a single letter:"

            input_tokens += count_tokens(prompt)
            
            # print(prompt)
            # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
            
            
            # Add back later
            output = LM_generation(prompt, temperature = 0.5, max_tokens = 3, model = model)
            
            # Printouts - Remove later:
            output = output[0]
            reasoning_content = output.choices[0].message.reasoning_content
            llm_answer = output.choices[0].message.content
            llm_answer = llm_answer[0]
            # print(reasoning_content)
            # print(content)
            
            bigtom_task3_ans[i]['reasoning_content'][k] = reasoning_content
            bigtom_task3_ans[i]['llm_answer'][k] = llm_answer

                    
            output_tokens += count_tokens(llm_answer)
            save_jsonl(bigtom_task3_ans, f'bigtom_task3_ans_{model}.jsonl')
            # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                    
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)


bigtom_task3_llm_answer = [instance['llm_answer'] for instance in bigtom_task3_ans]
bigtom_task3_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in bigtom_task3_ans]
# bigtom_task3_llm_answer = bigtom_task3_llm_answer[0:50]
# bigtom_task3_true_answer = bigtom_task3_true_answer[0:50]
bigtom_task3_llm_answer = np.array(bigtom_task3_llm_answer)
bigtom_task3_true_answer = np.array(bigtom_task3_true_answer)
bigtom_task3_accuracy = (bigtom_task3_llm_answer == bigtom_task3_true_answer).mean(axis=1)
gt_id = {'positive':0, 'hard_negative': 1}
bigtom_task3_correct = np.zeros((100, 2, 8)) # groundtruth: pos, hard_neg, easy_neg
bigtom_task3_total = np.zeros((100, 2, 8))
for i, instance in enumerate(bigtom_task3_ans):
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    bigtom_task3_total[story_idx][gt_idx][sense_idx] = 1
    bigtom_task3_correct[story_idx][gt_idx][sense_idx] = bigtom_task3_accuracy[i]
eps = 1e-10
print("BigToM Task 3 TPR, TNR-hard: ", bigtom_task3_correct.sum(axis = 0).sum(axis = 1)/(bigtom_task3_total.sum(axis = 0).sum(axis = 1) + eps))

# BigToM first - Task 3
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# for i in range(2):
for i in range(len(opentom_task3_ans)): # set to len(bigtom_task3_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(4):
        #story, event, protag_name, percept, initial_belief
        for j in range(2):# percept_order
            story = final_opentom_with_noise[i]['story']
            protag_name = final_opentom_with_noise[i]['protag_name']
            percept_signal = final_opentom_with_noise[i]['percept_cue']
            # in each iteration, pick a different noise
            percept_noise = final_opentom_with_noise[i]['noise'][k]
            
            if (j==0):
                percept = f"\n1. {percept_signal} \n2. {percept_noise}\n"
            else:
                percept = f"\n1. {percept_noise} \n2. {percept_signal}\n"
            
            
            initial_belief = final_opentom_with_noise[i]['initial_belief']
            prompt = opentom_system.format(story = story, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
            
            prompt += opentom_q1[k].format(protag_name = protag_name, initial_belief = initial_belief)
            prompt += "\nProvide the answer as a single letter:"

            input_tokens += count_tokens(prompt)
            
            # print(prompt)
            # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
            
            
            # Add back later
            output = LM_generation(prompt, temperature = 0.5, max_tokens = 3, model = model)
            
            # Printouts - Remove later:
            output = output[0]
            reasoning_content = output.choices[0].message.reasoning_content
            llm_answer = output.choices[0].message.content
            llm_answer = llm_answer[0]
            # print(reasoning_content)
            # print(content)
            
            opentom_task3_ans[i]['reasoning_content'][k] = reasoning_content
            opentom_task3_ans[i]['llm_answer'][k] = llm_answer

                    
            output_tokens += count_tokens(llm_answer)
            save_jsonl(opentom_task3_ans, f'opentom_task3_ans_{model}.jsonl')
            # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                    
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)



opentom_task3_llm_answer = [instance['llm_answer'] for instance in opentom_task3_ans]
opentom_task3_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in opentom_task3_ans]
# bigtom_task3_llm_answer = bigtom_task3_llm_answer[0:50]
# bigtom_task3_true_answer = bigtom_task3_true_answer[0:50]
opentom_task3_llm_answer = np.array(opentom_task3_llm_answer)
opentom_task3_true_answer = np.array(opentom_task3_true_answer)
opentom_task3_accuracy = (opentom_task3_llm_answer == opentom_task3_true_answer).mean(axis=1)
gt_id = {'positive':0, 'hard_negative': 1}
opentom_task3_correct = np.zeros((121, 2, 8)) # groundtruth: pos, hard_neg, easy_neg
opentom_task3_total = np.zeros((121, 2, 8))
for i, instance in enumerate(opentom_task3_ans):
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    opentom_task3_total[story_idx][gt_idx][sense_idx] = 1
    opentom_task3_correct[story_idx][gt_idx][sense_idx] = opentom_task3_accuracy[i]
eps = 1e-10
print("OpenToM Task 3 TPR, TNR-hard: ", opentom_task3_correct.sum(axis = 0).sum(axis = 1)/(opentom_task3_total.sum(axis = 0).sum(axis = 1) + eps))

0-th story
Time taken for these 5 stories: 346.8881878852844 seconds.
1-th story


Task 4

In [18]:
bigtom_task4_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X','X','X','X','X','X'],
                     'true_answer': [],
                     'true_answer_og': option_dict[instance['new_belief_label']],
                     'reasoning_content': ['X','X','X','X','X','X','X','X','X','X']} 
                    for instance in final_bigtom_with_noise]
opentom_task4_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X','X','X','X','X','X'],
                     'true_answer': [],
                     'true_answer_og': option_dict[instance['new_belief_label']],
                     'reasoning_content': ['X','X','X','X','X','X','X','X','X','X']}  
                    for instance in final_opentom_with_noise]

In [19]:
import random
random.seed(0)
np.random.seed(0)
task4_ans = [bigtom_task4_ans, opentom_task4_ans]
for t in task4_ans:
    for instance in t:
        for _ in range(5):
            randomizer = np.random.choice(np.arange(4), 4, replace=False) # 0 --> a0, 1 --> a1
            instance['answer_order'].append(randomizer.tolist())
            # instance['true_answer'].append(['A','B','C','D'][randomizer[instance['true_answer_og']]])
            true_answer_idx = np.where(randomizer == instance['true_answer_og'])[0][0]
            instance['true_answer'].append(['A','B','C','D'][true_answer_idx])

In [20]:
for t in task4_ans:
    for instance in t:
        instance['answer_order'] = instance['answer_order'] + instance['answer_order']
        instance['true_answer'] = instance['true_answer'] + instance['true_answer']
        

In [21]:
with open('opentom_task4_ans_deepseek-reasoner.jsonl') as f: # ambiguous story
    opentom_task4_ans = [json.loads(line) for line in f]

In [ ]:
# BigToM first - Task 4
model = "deepseek-reasoner"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# # for i in range(20,50):
for i in range(len(bigtom_task4_ans)): 
    print(f'{i}-th story')
    for k in range(3): # only look at first 3
        #story, event, protag_name, percept, initial_belief
        for j in range(2):
            story = final_bigtom_with_noise[i]['story']
            event = final_bigtom_with_noise[i]['event']
            protag_name = final_bigtom_with_noise[i]['protag_name']
            percept_signal = final_bigtom_with_noise[i]['percept_cue']
            # in each iteration, pick a different noise
            percept_noise = final_bigtom_with_noise[i]['noise'][k]
            
            if (j==0):
                percept = f"\n1. {percept_signal} \n2. {percept_noise}\n"
            else:
                percept = f"\n1. {percept_noise} \n2. {percept_signal}\n"
                
            initial_belief = final_bigtom_with_noise[i]['initial_belief']
            prompt = bigtom_system.format(story = story, event = event, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
            
            default_question = final_bigtom_with_noise[i]['default_question']
            randomizer = np.array(bigtom_task4_ans[i]['answer_order'][k])
            # print(final_bigtom[i]['answer_options'])
            # print(randomizer)
            first = final_bigtom_with_noise[i]['answer_options'][randomizer[0]]
            second = final_bigtom_with_noise[i]['answer_options'][randomizer[1]]
            third = final_bigtom_with_noise[i]['answer_options'][randomizer[2]]
            fourth = final_bigtom_with_noise[i]['answer_options'][randomizer[3]]
            prompt += bigtom_q2_prompt.format(default_question=default_question, first=first, second=second, third=third, fourth=fourth)
            prompt += "\nProvide the answer as a single letter:"

            input_tokens += count_tokens(prompt)
            
            # print(prompt)
            # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
            
            
            # Add back later
            output = LM_generation(prompt, temperature = 0.5, max_tokens = 3, model = model)
            
            # Printouts - Remove later:
            output = output[0]
            reasoning_content = output.choices[0].message.reasoning_content
            llm_answer = output.choices[0].message.content
            llm_answer = llm_answer[0]
            # print(reasoning_content)
            # print(content)
            
            bigtom_task4_ans[i]['reasoning_content'][k + 5*j] = reasoning_content
            bigtom_task4_ans[i]['llm_answer'][k + 5*j] = llm_answer

                    
            output_tokens += count_tokens(llm_answer)
            
            save_jsonl(bigtom_task4_ans, f'bigtom_task4_ans_{model}.jsonl')
            # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                    
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)


bigtom_task4_llm_answer = [instance['llm_answer'] for instance in bigtom_task4_ans]
bigtom_task4_true_answer = [instance['true_answer'] for instance in bigtom_task4_ans]
bigtom_task4_llm_answer = bigtom_task4_llm_answer # remove slicing later
bigtom_task4_true_answer = bigtom_task4_true_answer # remove slicing later
bigtom_task4_llm_answer = np.array(bigtom_task4_llm_answer)
bigtom_task4_true_answer = np.array(bigtom_task4_true_answer)
bigtom_task4_accuracy = (bigtom_task4_llm_answer == bigtom_task4_true_answer).mean(axis=1)


gt_id = {'positive':0, 'hard_negative': 1}
bigtom_task4_correct = np.zeros((100, 2, 8)) # groundtruth: pos, hard_neg, easy_neg
bigtom_task4_total = np.zeros((100, 2, 8))
for i, instance in enumerate(bigtom_task4_ans): # remove slicing later
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    bigtom_task4_total[story_idx][gt_idx][sense_idx] = 1
    bigtom_task4_correct[story_idx][gt_idx][sense_idx] = bigtom_task4_accuracy[i]
eps = 1e-10
print("BigToM Task 4 TPR, TNR-hard: ", bigtom_task4_correct.sum(axis = 0).sum(axis = 1)/(bigtom_task4_total.sum(axis = 0).sum(axis = 1) + eps))


# BigToM first - Task 1
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# for i in range(2):
for i in range(222,len(opentom_task4_ans)): # set to len(bigtom_task1_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(3):
        #story, event, protag_name, percept, initial_belief
        for j in range(2):
            story = final_opentom_with_noise[i]['story']
            protag_name = final_opentom_with_noise[i]['protag_name']
            percept_signal = final_opentom_with_noise[i]['percept_cue']
            # in each iteration, pick a different noise
            percept_noise = final_opentom_with_noise[i]['noise'][k]
            
            if (j==0):
                percept = f"\n1. {percept_signal} \n2. {percept_noise}\n"
            else:
                percept = f"\n1. {percept_noise} \n2. {percept_signal}\n"
                
            initial_belief = final_opentom_with_noise[i]['initial_belief']
            prompt = opentom_system.format(story = story, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
            
            default_question = final_opentom_with_noise[i]['default_question']
            randomizer = np.array(opentom_task4_ans[i]['answer_order'][k])
            # print(final_bigtom[i]['answer_options'])
            # print(randomizer)
            first = final_opentom_with_noise[i]['answer_options'][randomizer[0]]
            second = final_opentom_with_noise[i]['answer_options'][randomizer[1]]
            third = final_opentom_with_noise[i]['answer_options'][randomizer[2]]
            fourth = final_opentom_with_noise[i]['answer_options'][randomizer[3]]
            prompt += opentom_q2_prompt.format(default_question=default_question, first=first, second=second, third=third, fourth=fourth)
            
            prompt += "\nProvide the answer as a single letter:"

            input_tokens += count_tokens(prompt)
            
            # print(prompt)
            # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
            
            
            # Add back later
            output = LM_generation(prompt, temperature = 0.5, max_tokens = 3, model = model)
            
            # Printouts - Remove later:
            output = output[0]
            reasoning_content = output.choices[0].message.reasoning_content
            llm_answer = output.choices[0].message.content
            llm_answer = llm_answer[0]
            # print(reasoning_content)
            # print(content)
            
            opentom_task4_ans[i]['reasoning_content'][k + 5*j] = reasoning_content
            opentom_task4_ans[i]['llm_answer'][k + 5*j] = llm_answer

                    
            output_tokens += count_tokens(llm_answer)
            save_jsonl(opentom_task4_ans, f'opentom_task4_ans_{model}.jsonl')
            # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                    
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)



opentom_task4_llm_answer = [instance['llm_answer'] for instance in opentom_task4_ans]
opentom_task4_true_answer = [instance['true_answer'] for instance in opentom_task4_ans]
opentom_task4_llm_answer = opentom_task4_llm_answer # remove slicing later
opentom_task4_true_answer = opentom_task4_true_answer # remove slicing later
opentom_task4_llm_answer = np.array(opentom_task4_llm_answer)
opentom_task4_true_answer = np.array(opentom_task4_true_answer)
opentom_task4_accuracy = (opentom_task4_llm_answer == opentom_task4_true_answer).mean(axis=1)


gt_id = {'positive':0, 'hard_negative': 1}
opentom_task4_correct = np.zeros((121, 2, 8)) # groundtruth: pos, hard_neg, easy_neg
opentom_task4_total = np.zeros((121, 2, 8))
for i, instance in enumerate(opentom_task4_ans): # remove slicing later
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    opentom_task4_total[story_idx][gt_idx][sense_idx] = 1
    opentom_task4_correct[story_idx][gt_idx][sense_idx] = opentom_task4_accuracy[i]
eps = 1e-10
print("OpenToM Task 4 TPR, TNR-hard: ", opentom_task4_correct.sum(axis = 0).sum(axis = 1)/(opentom_task4_total.sum(axis = 0).sum(axis = 1) + eps))


222-th story
223-th story
224-th story
225-th story
Time taken for these 5 stories: 1130.5392591953278 seconds.
226-th story
227-th story
228-th story
229-th story
230-th story
Time taken for these 5 stories: 1254.8644890785217 seconds.
231-th story
232-th story
233-th story
234-th story
235-th story
Time taken for these 5 stories: 1200.3369221687317 seconds.
236-th story
237-th story
238-th story
239-th story
240-th story
Time taken for these 5 stories: 910.0661590099335 seconds.
241-th story
242-th story
243-th story
244-th story
245-th story
Time taken for these 5 stories: 588.7686750888824 seconds.
246-th story
247-th story
248-th story
249-th story
250-th story
Time taken for these 5 stories: 1057.615769147873 seconds.
251-th story
252-th story
253-th story
254-th story
255-th story
Time taken for these 5 stories: 553.3187613487244 seconds.
256-th story
257-th story
258-th story
259-th story
260-th story
Time taken for these 5 stories: 798.5960559844971 seconds.
261-th story
262-t